<a href="https://colab.research.google.com/github/JuliaKovalchook/Ukraine-TopoJSON/blob/main/Instructions_for_creating_%D1%81ontour_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Openstreetmap

Openstreetmap містить інформацію щодо координат і багато іншої географічної інформації щодо всіх країн, громад, сіл і навіть озер чи ставків.
Існує як і API для витягу даних так і платформа, яка одразу допомагає візуалізувати запит для витягу дані.  
<br>
Якщо ви бажаєте витягнути лише декількох файлів з інформацією або ви ще не надто впевнено себе відчуваєте з API, Overpass turbo є найкращим рішенням оскільки візуалізація одразу показує які дані є зайві, а яких не хватає.


Ми розглянемо використання двох інструментів:

1.   [Overpass turbo](https://overpass-turbo.eu/)
2.   Overpass API    ([документація](https://wiki.openstreetmap.org/wiki/Overpass_API))






### Overpass turbo

#### Інструкція

Приклад коду який витягуватиме дані всіх громад Київська області


```
[out:json][timeout:25];  
area['admin_level'='4']['name:uk'='Київська область'];
(rel['admin_level'='7'](area););
out geom;
```


Перший рядок `[out:json]` вказує дані якого формату ми бажаємо витягнути <br>

Також можливі варінати витяг даних формату xml `[out:xml]` чи csv, при цьому для файлів формату csv обовязково вказувати теги, які ми бажаємо витягнути `[out:csv(::type,::id,name,"name:en")]` та [інші](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL#Output_format_(out:)


`[timeout:25]` обмежує час виконання запиту до 25 секунд. Якщо щось піде не так, то ліміт гарантує, що сервер не витрачатиме час.


---

Другий рядок `area['admin_level'='4']['name:uk'='Київська область']` фільтрує дані вказуючи, що ми хочемо отримати  дані, які знаходяться в межах  де admin_level=4 (тобто область) і називається "Київська область". Якщо ми бажаємо отримати дані по всій країні, то варто змінити рядок на `area['admin_level'='2']['name:uk'='Україна']`

---

Третій рядок `(rel['admin_level'='7'](area);)` вказує що ми хочемо отримати, в даному випадку це `rel` (відносини), інші можливі ваіранти це `node` і `way`. Географічна інформація про широту/довжину завжди зберігається у вузлах. Області з понад 2000 вузлів не можуть бути представлені шляхом, ця функція вимагатиме складнішої багатополігональної структури відсини, тому в даному випадку використовується саме сутність rel (відносини). Проте сутність відносини повертує також "сусідів" нашого шуканого запиту, про це детальніше в розділі "випавлення помилок" <br> `'admin_level'='7'` вкзаує, що саме ми хочемо витягнути села, міста, райони, в даному випадку це громади.

---

Нарешті, `out geom` виводить відношення (включаючи їх дочірні шляхи та вузли). Існують багато інших варіантів оператору виходу, вони гарно описані [тут](https://dev.overpass-api.de/overpass-doc/en/targets/formats.html)


#### Деякі приклади

Приклад коду який витягуватиме дані всіх областей України

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='4'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх районів України* (після реформи децентралізації)

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='6'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх районів України* (до реформи децентралізації)
Вказуючи дату ми отримуємо карту, яка існувала на момент зазначений в коді, майте на увазі це значно сповільнює виконання коду.

```
[date:"2014-01-01T19:00:00Z"]
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='6'](area););
out geom;

```

Приклад коду який витягуватиме дані всіх громад України*

```
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='7'](area););
out geom;
```

Приклад коду який витягуватиме дані всіх районів Києва

```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='10'](area););
out geom;
```
**(за виключенням тимчасово окупованих тереторій)*


---




#### Виправлення даних

Для тестування використаємо функцію `[bbox]`, яка повертатиме дані лише в
 виділеній області `[bbox:{{bbox}}]` або в зазначених координатах `[bbox:49.560852, 22.617202, 50.659908, 24.490371]`

####Помилка "сусіди"

(не закінчений розділ!!)




Очікувано, що запит нижче поверне координати міста Києва, але він поверне також Київську область.
```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4'](area););
out geom;
```
Проблема в тому, що запит повертатиме шляхи, які є частинами *relative* міста Київа, оскільки Київ межує з Київсьою областю (тобтот має спільні шляхи) і Київська область теж має `'admin_level'='4'`, тому ми отримуємо дві сутності Київ та Київська область.  Ця помилка може вирішитися специфікацією, що ми шукаємо саме місто, якщо ці сутності мають одинаковий тип, наприклад в випадку прикордонних областей, ми можемо відфільтрувати їх за іншим тегами, напрклад для області вказати `["iso3166-2"~"^UA-"]` де для позначення України використовується  *UA* або для районі (до реформи централізації) `['wikipedia'~ '^uk']`



Місто Київ не є громадою, проте якщо ми витягуватимео дані по всіх громадах Київської області, то на мапі України в нас буде пусте місце яке належитеме місту Києву. Отже припустимо ми бажаємо отримати всі громади Київської області і міста Київ. Оскільки admin_level для міста Києва і області одинаковий, то варто вказати, що ми шукаємо саме місто  `['place'='city']`

```
[out:json];
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4']['place'='city'](area););
out geom;
```

Після цього, якщо запити не суперечать одне одному ми можемо їх просто обєднати
```
[out:json];
area['admin_level'='4']['name:uk'='Київська область'];
(rel['admin_level'='7'](area););
out geom;
area['admin_level'='4']['name:uk'='Київ'];
(rel['admin_level'='4']['place'='city'](area););
out geom;
```








Крим це Україна. Проте, на жаль, в Криму не відбулася реформат децентралізації, тому в ньому і немає районів з `admin_level'='7'` припустимо, що ми бажаємо отримати карту громад України і райони Криму до окупації. <br>
Запит для отримання громад повертає близько 70 мб, для тестування нашого коду обмежемо область лише видимою картою додавши `bbox:{{bbox}}`, також нас не цікавлять зміни адміністративних кордонів після окупації, тому задамо параметр   `[date:"2014-01-01T19:00:00Z"]`, який повертає стан мапи в зафіксовний період.

```
[bbox:{{bbox}}]
[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(rel['admin_level'='7'](area););
out geom;
```

Оскільки


### Витяг даних

Якщо ми використовуємо Overpass turbo, то варто натиснути "Експорт" -> "GeoJSON"


1.   Пункт списку
2.   Пункт списку



/*[bbox:44.268805, 32.124023, 46.672056, 36.469116]
[out:json];
(rel['admin_level'='7'](area););
out geom;
area['admin_level'='4']['name:uk'='Автономна Республіка Крим'];
(rel['admin_level'='6'](area););
out geom;
area['admin_level'='4']['name:uk'='Севастополь'];
(rel['admin_level'='4'](area););
out geom;


[out:json];
area['admin_level'='4']['name:uk'='Автономна Республіка Крим'];
(rel['admin_level'='6'](area)(newer:"2020-04-01T00:00:00Z"););
out geom;



[date:"2013-01-01T19:00:00Z"]
[out:json];
area['admin_level'='2']['name:uk'='Україна']->.country;
area['admin_level'='4']['name:uk'='Луганська область'](area.country)->.region;
(node['admin_level'='4'](area.region););
out geom ;
*/

[out:json];
area['admin_level'='2']['name:uk'='Україна'];
(relation['admin_level'='6'](area););
out geom;
